In [1]:
from tqdm import tqdm
import numpy as np
import os, glob
import shutil
import matplotlib.pyplot as plt
import cv2
from copy import deepcopy

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch import optim
from torch.nn import functional as F

import sys
sys.path.append('/raid-dgx1/Hasnat/Covoiturage/')
from my_utilities_display_related import show_image, show_image_from_file, show_image_subplots

os.environ["CUDA_VISIBLE_DEVICES"]="1"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print((torch.cuda.is_available(), torch.version.cuda)) 

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

(True, '10.1')


In [2]:
from dataset import Yolo_dataset
from tool.tv_reference.utils import collate_fn as val_collate

from cfg import Cfg
import argparse
from easydict import EasyDict as edict

def get_args(**kwargs):
    cfg = kwargs
    parser = argparse.ArgumentParser(description='Train the Model on images and target masks',
                                     formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    # parser.add_argument('-b', '--batch-size', metavar='B', type=int, nargs='?', default=2,
    #                     help='Batch size', dest='batchsize')
    parser.add_argument('-l', '--learning-rate', metavar='LR', type=float, nargs='?', default=0.001,
                        help='Learning rate', dest='learning_rate')
    parser.add_argument('-f', '--load', dest='load', type=str, default=None,
                        help='Load model from a .pth file')
    parser.add_argument('-g', '--gpu', metavar='G', type=str, default='-1',
                        help='GPU', dest='gpu')
    parser.add_argument('-dir', '--data-dir', type=str, default=None,
                        help='dataset dir', dest='dataset_dir')
    parser.add_argument('-pretrained', type=str, default=None, help='pretrained yolov4.conv.137')
    parser.add_argument('-classes', type=int, default=80, help='dataset classes')
    parser.add_argument('-train_label_path', dest='train_label', type=str, default='train.txt', help="train label path")
    parser.add_argument(
        '-optimizer', type=str, default='adam',
        help='training optimizer',
        dest='TRAIN_OPTIMIZER')
    parser.add_argument(
        '-iou-type', type=str, default='iou',
        help='iou type (iou, giou, diou, ciou)',
        dest='iou_type')
    parser.add_argument(
        '-keep-checkpoint-max', type=int, default=10,
        help='maximum number of checkpoints to keep. If set 0, all checkpoints will be kept',
        dest='keep_checkpoint_max')
    args = vars(parser.parse_args())

    # for k in args.keys():
    #     cfg[k] = args.get(k)
    cfg.update(args)

    return edict(cfg)

def collate(batch):
    images = []
    bboxes = []
    for img, box in batch:
        images.append([img])
        bboxes.append([box])
    images = np.concatenate(images, axis=0)
    images = images.transpose(0, 3, 1, 2)
    images = torch.from_numpy(images).div(255.0)
    bboxes = np.concatenate(bboxes, axis=0)
    bboxes = torch.from_numpy(bboxes)
    return images, bboxes

In [3]:
cfg = get_args(**Cfg)

config = cfg
cfg.batch = 16
train_dataset = Yolo_dataset(config.train_label, config, train=True)
val_dataset = Yolo_dataset(config.val_label, config, train=False)

n_train = len(train_dataset)
n_val = len(val_dataset)


train_loader = DataLoader(train_dataset, batch_size=config.batch // config.subdivisions, shuffle=True,
                          num_workers=16, pin_memory=False, drop_last=True, collate_fn=collate)


In [4]:
cfg

{'use_darknet_cfg': True,
 'cfgfile': '/raid-dgx1/Hasnat/pytorch-YOLOv4/cfg/yolov4.cfg',
 'batch': 16,
 'subdivisions': 16,
 'width': 608,
 'height': 608,
 'channels': 3,
 'momentum': 0.949,
 'decay': 0.0005,
 'angle': 0,
 'saturation': 1.5,
 'exposure': 1.5,
 'hue': 0.1,
 'learning_rate': 0.001,
 'burn_in': 1000,
 'max_batches': 500500,
 'steps': [400000, 450000],
 'policy': [400000, 450000],
 'scales': [0.1, 0.1],
 'cutmix': 0,
 'mosaic': 1,
 'letter_box': 0,
 'jitter': 0.2,
 'classes': 80,
 'track': 0,
 'w': 608,
 'h': 608,
 'flip': 1,
 'blur': 0,
 'gaussian': 0,
 'boxes': 60,
 'TRAIN_EPOCHS': 300,
 'train_label': 'train.txt',
 'val_label': '/raid-dgx1/Hasnat/pytorch-YOLOv4/data/val.txt',
 'TRAIN_OPTIMIZER': 'adam',
 'mixup': 3,
 'checkpoints': '/raid-dgx1/Hasnat/pytorch-YOLOv4/checkpoints',
 'TRAIN_TENSORBOARD_DIR': '/raid-dgx1/Hasnat/pytorch-YOLOv4/log',
 'iou_type': 'iou',
 'keep_checkpoint_max': 10,
 'load': '/home/mhasnat/.local/share/jupyter/runtime/kernel-8e8c6c9a-ad42-43a8-b

In [ ]:
for i, batch in train_loader:
    print(i, batch)
    break

In [ ]:
i

In [ ]:
len(train_dataset.truth)

In [ ]:
for images, targets, _ in data_loader_train:
    images = list(img.to(device) for img in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    targets = [modify_target(t_, device) for t_ in targets]
    break